# 📈 AI Trading Agent for Binance - Proof of Concept

This notebook demonstrates a basic AI trading agent that:
- Fetches your portfolio and market data using the Binance API
- Applies a basic rule-based strategy to decide investments
- (In full deployment) Places orders if expected ROI > 5%

> Note: This is a PoC. Use testnet or paper trading before going live.

In [ ]:
# 📦 Install necessary packages
!pip install python-binance pandas numpy matplotlib

In [ ]:
# 🔐 Setup API keys securely
from binance.client import Client
import os

# Replace with your own keys or use dotenv for security
api_key = os.getenv('BINANCE_API_KEY', 'your_api_key_here')
api_secret = os.getenv('BINANCE_API_SECRET', 'your_api_secret_here')
client = Client(api_key, api_secret)

In [ ]:
# 📊 Fetch account balances
account_info = client.get_account()
balances = [b for b in account_info['balances'] if float(b['free']) > 0]
for b in balances:
    print(f"{b['asset']}: {b['free']}")

In [ ]:
# ⏳ Get historical price data (OHLCV)
import pandas as pd
import datetime

def get_klines(symbol='BTCUSDT', interval='1d', lookback='30 day ago UTC'):
    klines = client.get_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
        'quote_asset_volume', 'number_of_trades', 'taker_buy_base_volume',
        'taker_buy_quote_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close']].astype(float)
    return df

btc_df = get_klines()
btc_df.tail()

In [ ]:
# 🧠 Basic ROI > 5% rule-based strategy
df = btc_df.copy()
df['prev_close'] = df['close'].shift(1)
df['roi'] = (df['close'] - df['prev_close']) / df['prev_close']
df['signal'] = df['roi'].apply(lambda x: 'BUY' if x > 0.05 else 'HOLD')
df.tail()

In [ ]:
# 📈 Plot the close price with buy signals
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label='Close Price')
plt.scatter(df[df['signal'] == 'BUY'].index, df[df['signal'] == 'BUY']['close'], 
            label='Buy Signal', color='green', marker='^')
plt.title('BTCUSDT Price & Buy Signals')
plt.legend()
plt.grid()
plt.show()